In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

from tqdm.auto import tqdm

from simulator import *

In [3]:
cache_size = 30
catalog_size = 100
sample_size_day = 4000

trace_day1 = gen_irm_trace(sample_size=sample_size_day, catalog_size=catalog_size, power_law_exp=0.8, shuffled=True)
trace_day2 = gen_irm_trace(sample_size=sample_size_day, catalog_size=catalog_size, power_law_exp=0.4, shuffled=True)
trace_day3 = np.random.randint(low=0, high=catalog_size-1, size=sample_size_day).tolist()
trace_day4 = gen_irm_trace(sample_size=sample_size_day, catalog_size=catalog_size, power_law_exp=0.7, shuffled=True)
trace_day5 = gen_irm_trace(sample_size=sample_size_day, catalog_size=catalog_size, power_law_exp=0.5, shuffled=True)
trace_day6 = gen_irm_trace(sample_size=sample_size_day, catalog_size=catalog_size, power_law_exp=0.9, shuffled=True)


traces = [trace_day1, trace_day2, trace_day3, trace_day4, trace_day5]

switching_points = np.cumsum([len(t) for t in traces])-1 # -1 to compensate for the index starting at 0

# Able to switch twice a day
# switching_points = np.cumsum([sample_size_day // 2 for _ in range(len(traces)*2)])-1

trace = [r for t in traces for r in t]
sample_size = len(trace)
# switching_points

In [4]:
cache_init = init_cache(cache_size, catalog_size)

cache_LRU = LRU(cache_size, catalog_size, cache_init)
cache_LFU = LFU(cache_size, catalog_size, cache_init)
cache_BH = CacheStatic(cache_size, catalog_size, gen_best_static(trace, cache_size))
cache_OGD = OGD(cache_size, catalog_size, sample_size)
cache_FTPL = FTPL(cache_size, catalog_size, cache_init)

OGD_init = lambda cache_size, catalog_size, cache_init: OGD(cache_size, catalog_size, sample_size)
experts = (LRU, LFU, OGD_init, FTPL)


cache_EP = ExpertsCacheNeq(cache_size, catalog_size, cache_init, experts, init_rnd_expert=False)
cache_EP.name = "Cache EP"
cache_EP_island = ExpertsCacheNeq(cache_size, catalog_size, cache_init, experts, init_rnd_expert=False)
cache_EP_island.name = "EP Island"
cache_EP_island_wr = ExpertsCacheNeq(cache_size, catalog_size, cache_init, experts, init_rnd_expert=False)
cache_EP_island_wr.name = "EP Island + WR"


for i, request in tqdm(enumerate(trace), total=len(trace)):
    cache_LRU.request(request)
    cache_LFU.request(request)
    cache_BH.request(request)
    cache_OGD.request(request)
    cache_FTPL.request(request)
    cache_EP.request(request)
    cache_EP_island.request(request, switch=i in switching_points, weight_reset=False)
    cache_EP_island_wr.request(request, switch=i in switching_points, weight_reset=True)

In [9]:
%matplotlib ipympl
plot_comp(\
          cache_LRU,\
          cache_LFU,\
          cache_BH,\
          cache_OGD,\
          cache_FTPL,\
#           cache_EP,\
          cache_EP_island,\
          cache_EP_island_wr,\
         )
plt.vlines(switching_points, ymin=0, ymax=1.0, linestyles="dashed")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def get_best_caches(caches, points):
    best_caches = []
    for p in points:
        hits = [cache.get_hitrate()[p] for  cache in caches]
        best_cache = caches[hits.index(max(hits))]
        best_caches.append(best_cache.get_name())
    return best_caches

In [7]:
best_caches = get_best_caches([cache_LRU, cache_LFU, cache_OGD, cache_FTPL], switching_points)

EP_choices = [cache_EP.expert_choices[sp-1] for sp in switching_points]
EP_island_choices = [cache_EP_island.expert_choices[sp-1] for sp in switching_points]
EP_island_choices_wr = [cache_EP_island_wr.expert_choices[sp-1] for sp in switching_points]

In [8]:
print("EP:\t\t", EP_choices)
print("EP_island:\t", EP_island_choices)
print("EP_island_wr:\t", EP_island_choices_wr)
print("Best caches:\t", best_caches)
# Note: FTPL and LFU have very close performance and depending on the randomness one or the other is better

EP:		 ['LFU', 'LFU', 'LFU', 'OGD', 'OGD']
EP_island:	 ['LRU', 'LFU', 'LFU', 'LFU', 'OGD']
EP_island_wr:	 ['LRU', 'LFU', 'OGD', 'LRU', 'OGD']
Best caches:	 ['FTPL', 'FTPL', 'LFU', 'OGD', 'OGD']
